Madusanka Madiligama
2/5/2024

In [23]:
import pandas as pd
import numpy as np 

import geopandas as gpd
import pandas as pd
import plotly.express as px
import json

import warnings
warnings.filterwarnings('ignore')

fields_to_read = ['company_name','legal_name','address1','city','state','zip5','zip4','address_type_code','mail_addr_address1','mail_addr_city','mail_addr_state','mail_addr_zip5','mail_addr_zip4','mail_addr_address_type_code','county_code','county_name','cbsa','cbsa_name','dma','dma_name','latitude','longitude','census_id','census_block','census_tract','MedianIncomeCensusArea','MeanHousingCensusArea','url','naics_desc','sic2code','sic4code','sic6code','sic2desc','sic4desc','sic6desc','sic8desc','sic_division','exact_sales_volume','sales_volume','exact_number_of_employees','number_of_employees','employee_code','location_type','parent_company','parent_address','parent_city','parent_state','parent_zip','business_specialty','company_year_started','business_type','state_where_entity_formed','minority','woman','government','small','home_office','franchise','chain','site_status','zip_centroid_lat','zip_centroid_long','owner_company','owner_address','owner_city','owner_state','owner_zip','owner_country','owner_phone']
df_newspaper = pd.read_csv('../artifacts/data/Raw/newspaper.csv', quotechar='"', sep=',', on_bad_lines='skip', usecols=fields_to_read)
df_radio = pd.read_csv('../artifacts/data/Raw/radio.csv', quotechar='"', sep=',', on_bad_lines='skip', usecols=fields_to_read)
df_television = pd.read_csv('../artifacts/data/Raw/television.csv', quotechar='"', sep=',', on_bad_lines='skip', usecols=fields_to_read)
df_shapefile_NatDMA = gpd.read_file('../artifacts/data/shapefiles/NatDMA.shp')
df_shapefile_county = gpd.read_file('../artifacts/data/shapefiles/cb_2018_us_county_500k.shp')
df_county = pd.read_csv('../artifacts/data/shapefiles/county.csv')

In [24]:
def add_geography_column(df_media, df_county):
    # Merge to get the 'Geography' column based on state and county name
    merged_df = df_media.merge(df_county, left_on=['state', 'county_name'], right_on=['New_State', 'New_County'])
    
    # If you want to keep only the original columns from df_media and the 'Geography' column from df_county
    merged_df = merged_df[df_media.columns.tolist() + ['Geography']]
    
    return merged_df

# Apply the function to each media DataFrame
df_newspaper_with_geography = add_geography_column(df_newspaper, df_county)
df_radio_with_geography = add_geography_column(df_radio, df_county)
df_television_with_geography = add_geography_column(df_television, df_county)

In [39]:
import pandas as pd
import plotly.express as px
import json

# Function to process the data and create the map
def process_and_plot(df_media, df_shapefile_county, media_type, output_filename):
    # Deduplicate the DataFrame based on certain fields
    fields_to_dedupe = ['company_name', 'address1']
    df_deduped_media = df_media.drop_duplicates(subset=fields_to_dedupe)
    
    # Select necessary columns and drop NaN values
    df_media_selected = df_deduped_media[["company_name", "latitude", "longitude", 'state', 'county_code', 'county_name', 'dma_name', 'dma', 'Geography']].dropna()
    
    # Merge with the county shapefile DataFrame based on the 'Geography'
    merged_gdf_county = df_shapefile_county.merge(df_media_selected, how='left', left_on='AFFGEOID', right_on='Geography')
    
    # Count DMA occurrences
    merged_gdf_county['station_count'] = merged_gdf_county.groupby('Geography')['Geography'].transform('count')
    
    # Remove duplicates based on 'Geography'
    merged_gdf_county = merged_gdf_county.drop_duplicates(subset=['Geography'])
    
    # Convert GeoDataFrame to JSON for the choropleth
    json_data = merged_gdf_county.geometry.to_json()
    
    # Convert GeoJSON to dictionary (which is compatible with plotly)
    geojson = json.loads(json_data)
    
    # Replace NaN values with a placeholder (-1) for color mapping
    merged_gdf_county['station_count'].fillna(-1, inplace=True)
    
    # Create a choropleth mapbox
    fig = px.choropleth_mapbox(
        merged_gdf_county,
        geojson=geojson,
        locations=merged_gdf_county.index,
        color='station_count',
        color_continuous_scale="Viridis",
        range_color=(0, merged_gdf_county['station_count'].max()),
        mapbox_style="carto-positron",
        zoom=3,
        center={"lat": 37.0902, "lon": -95.7129},
        opacity=0.5,
        hover_name='county_name'
    )
    
    # Set color for NaN values using custom color scale
    fig.update_traces(
        below='',
        marker_line_width=0,
        marker_opacity=0.7,
        colorscale=[(0, "lightgrey"), (0.001, "blue"), (1, "red")],
        zmin=0,
        zmax=merged_gdf_county['station_count'].max(),
        cmin=-1,
        cmax=merged_gdf_county['station_count'].max()
    )
    
    # Save the figure as an HTML file
    fig.write_html(output_filename)

# Assuming df_newspaper_with_geography, df_shapefile_county are already defined
# Call the function for the newspaper media type
process_and_plot(df_newspaper_with_geography, df_shapefile_county, 'newspaper', '../artifacts/output/NewspaperStations_corrected_with_Null_counties.html')
# Repeat for other media types (df_radio_with_geography, df_television_with_geography, etc.) with the respective file paths.


ValueError: Invalid property specified for object of type plotly.graph_objs.Choroplethmapbox: 'cmin'

Did you mean "zmin"?

    Valid properties:
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `colorscale`. In case `colorscale` is unspecified or
            `autocolorscale` is true, the default palette will be
            chosen according to whether numbers in the `color`
            array are all positive, all negative or mixed.
        below
            Determines if the choropleth polygons will be inserted
            before the layer with the specified ID. By default,
            choroplethmapbox traces are placed above the water
            layers. If set to '', the layer will be inserted above
            every existing layer.
        coloraxis
            Sets a reference to a shared color axis. References to
            these shared color axes are "coloraxis"

In [34]:
# Define the function to process and create the map for each media type
def process_and_plot(df_media,  df_shapefile_county, media_type, output_filename):
    # Deduplicate the DataFrame based on certain fields
    fields_to_dedupe = ['company_name', 'address1']
    df_deduped_media = df_media.drop_duplicates(subset=fields_to_dedupe)
    
    # Select necessary columns and drop NaN values
    df_media_selected = df_deduped_media[["company_name", "latitude", "longitude", 'state','county_code','county_name','dma_name', 'dma','Geography']].dropna()
    df_media_selected = df_media_selected.dropna()
    # Merge with the county shapefile DataFrame based on the 'county_name'
    # merged_gdf_county = df_shapefile_county.merge(df_media_selected, left_on='AFFGEOID', right_on='Geography')
    # merged_gdf_county = df_shapefile_county.merge(df_media_selected, left_on='AFFGEOID', right_on='Geography')
    merged_gdf_county = df_shapefile_county.merge(df_media_selected, how='left', left_on='AFFGEOID', right_on='Geography')

    
    # Count DMA occurrences
    merged_gdf_county['station_count'] = merged_gdf_county.groupby('Geography')['Geography'].transform('count')
    
    # Remove duplicates based on 'county_name'
    merged_gdf_county = merged_gdf_county.drop_duplicates(subset=['Geography'])

    json_data = merged_gdf_county.geometry.to_json()

    # Convert GeoJSON to dictionary 
    geojson = json.loads(json_data)

    fig = px.choropleth_mapbox(
        merged_gdf_county,
        geojson=geojson,
        locations=merged_gdf_county.index,  # This should be the column in merged_gdf that matches the GeoJSON features
        color='station_count',
        color_continuous_scale="Viridis",
        mapbox_style="carto-positron",
        zoom=3,
        center={"lat": 37.0902, "lon": -95.7129},
        opacity=0.5,
        hover_name='county_name',  # This will be displayed as the title in the hover labels
        hover_data=None,  # We will define hover information manually using hovertemplate

    )

    # Set the hovertemplate to only include the information you want
    # fig.update_traces(
    #     hovertemplate="<b>%{hovertext}</b><br>Number of News stations: %{z}<extra></extra>"
    # )
    fig.update_traces(marker=dict(colorscale='Viridis', cmin=0, cmax=merged_gdf_county['station_count'].max(),
                              color=np.where(merged_gdf_county['station_count'].isnull(), 'lightgrey', merged_gdf_county['station_count'])))

    # Adjust color bar size and layout properties
    with fig.batch_update():
        fig.update_layout(coloraxis_colorbar=dict(
            title='Number of News stations in each county',
            thicknessmode="pixels", thickness=10,
            lenmode="pixels", len=200
        ))
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    # Save the figure as an HTML file
    fig.write_html(output_filename)

In [37]:
# Call the function for each media type
process_and_plot(df_newspaper_with_geography,  df_shapefile_county, 'newspaper', '../artifacts/output/NewspaperStations_corrected_with_Null_counties.html')
process_and_plot(df_radio_with_geography,  df_shapefile_county, 'radio', '../artifacts/output/RadioStations_corrected_with_Null_counties.html')
process_and_plot(df_television_with_geography,  df_shapefile_county, 'television', '../artifacts/output/TelevisionStations_corrected_with_Null_counties.html')

ValueError: Invalid property specified for object of type plotly.graph_objs.Choroplethmapbox: 'cmin'

Did you mean "zmin"?

    Valid properties:
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `colorscale`. In case `colorscale` is unspecified or
            `autocolorscale` is true, the default palette will be
            chosen according to whether numbers in the `color`
            array are all positive, all negative or mixed.
        below
            Determines if the choropleth polygons will be inserted
            before the layer with the specified ID. By default,
            choroplethmapbox traces are placed above the water
            layers. If set to '', the layer will be inserted above
            every existing layer.
        coloraxis
            Sets a reference to a shared color axis. References to
            these shared color axes are "coloraxis"